In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 64, 3)
        self.fc1 = nn.Linear(64*24*24, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x))

In [0]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
batch_size = 64
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
lr = 0.01
epochs = 3

data_loc = '/content/'

t = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root=data_loc, train=True, transform=t, download=True)
test_dataset = datasets.MNIST(root=data_loc, train=False, transform=t)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = BaseModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval=100)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299880
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.066614
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.493760
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.286462
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.327690
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.206194
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.296447
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.329398
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.177194
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.206989

Test set: Average loss: 0.1859, Accuracy: 9465/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.091135
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.350534
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.164717
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.205324
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.132031
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.248782
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.174042
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.230818
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.197198
T